In [1]:
import pandas as pd
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Lasso 

from src.paths import TRAINING_DATA
from src.hyperparameter_tuning import optimise_hyperparams

/home/kobina/.cache/pypoetry/virtualenvs/src-Frx8CZzu-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: xgboost, lightgbm, sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/kobina/divvy-bikes-demand-predictor-public/34428ae078294518b4797548c73ed10d



#### Starts

In [ ]:
start_table = pd.read_parquet(TRAINING_DATA/"start_table.parquet")

In [ ]:
best_lasso_hyperparams_for_starts = \
  optimise_hyperparams(
    model_fn=Lasso, 
    hyperparam_trials = 5, 
    scenario = "start", 
    X = start_table.drop("trips_next_hour", axis = 1), 
    y = start_table["trips_next_hour"]
)

### Stops

In [2]:
stop_table = pd.read_parquet(TRAINING_DATA/"stop_table.parquet") 

#### Lasso

In [ ]:
best_lasso_hyperparams_for_stops = \
  optimise_hyperparams(
    model_fn=Lasso, 
    hyperparam_trials = 5, 
    scenario = "stop", 
    X = stop_table.drop("trips_next_hour", axis = 1), 
    y = stop_table["trips_next_hour"]
)

#### LGBMRegressor

In [3]:
best_lgb_hyperparams_for_stops = \
  optimise_hyperparams(
    model_fn=LGBMRegressor, 
    hyperparam_trials=5, 
    scenario = "stop", 
    X=stop_table.drop("trips_next_hour", axis = 1),
    y=stop_table["trips_next_hour"]
)

divvy_trips - INFO - Beginning hyperparameter search
[I 2024-02-06 00:57:34,441] A new study created in memory with name: no-name-aa30187f-a3d6-42d1-b3f2-db02a5ec7ce1
divvy_trips - INFO - Start Trial 0
divvy_trips - INFO - Performing split number 0


divvy_trips - INFO - Performing split number 1
divvy_trips - INFO - Performing split number 2
divvy_trips - INFO - Performing split number 3
divvy_trips - INFO - Performing split number 4
[I 2024-02-06 00:58:10,871] Trial 0 finished with value: 0.20496896371584833 and parameters: {'num_leaves': 50, 'max_depth': 7, 'n_estimators': 52, 'learning_rate': 0.6653855922654952, 'importance_type': 'split', 'subsample': 0, 'feature_fraction': 0.8074421168630138, 'colsample_by_tree': 0.36967134549956615, 'colsample_by_level': 0.24751151073675703, 'colsample_by_node': 0.3286460741051326, 'bagging_fraction': 0.2842090588611806}. Best is trial 0 with value: 0.20496896371584833.
divvy_trips - INFO - Start Trial 1
divvy_trips - INFO - Performing split number 0


divvy_trips - INFO - Performing split number 1
divvy_trips - INFO - Performing split number 2
divvy_trips - INFO - Performing split number 3
divvy_trips - INFO - Performing split number 4
[I 2024-02-06 00:59:55,709] Trial 1 finished with value: 0.20672365820234306 and parameters: {'num_leaves': 58, 'max_depth': 8, 'n_estimators': 109, 'learning_rate': 0.5263831308238435, 'importance_type': 'gain', 'subsample': 0, 'feature_fraction': 0.5552746530779602, 'colsample_by_tree': 0.19430008653634756, 'colsample_by_level': 0.8855665348277181, 'colsample_by_node': 0.7546481091920707, 'bagging_fraction': 0.614286172764395}. Best is trial 0 with value: 0.20496896371584833.
divvy_trips - INFO - Start Trial 2
divvy_trips - INFO - Performing split number 0


divvy_trips - INFO - Performing split number 1
divvy_trips - INFO - Performing split number 2
divvy_trips - INFO - Performing split number 3
divvy_trips - INFO - Performing split number 4
[I 2024-02-06 01:02:03,510] Trial 2 finished with value: 0.2044856218795681 and parameters: {'num_leaves': 39, 'max_depth': 9, 'n_estimators': 138, 'learning_rate': 0.5090172649844095, 'importance_type': 'split', 'subsample': 0, 'feature_fraction': 0.5029190019298146, 'colsample_by_tree': 0.2622408641705394, 'colsample_by_level': 0.8916317127138547, 'colsample_by_node': 0.3928637682806727, 'bagging_fraction': 0.5270861842813163}. Best is trial 2 with value: 0.2044856218795681.
divvy_trips - INFO - Start Trial 3
divvy_trips - INFO - Performing split number 0


divvy_trips - INFO - Performing split number 1
divvy_trips - INFO - Performing split number 2
divvy_trips - INFO - Performing split number 3
divvy_trips - INFO - Performing split number 4
[I 2024-02-06 01:03:23,400] Trial 3 finished with value: 0.208052642421611 and parameters: {'num_leaves': 42, 'max_depth': 9, 'n_estimators': 100, 'learning_rate': 0.6255044339206903, 'importance_type': 'gain', 'subsample': 1, 'feature_fraction': 0.2720517991921456, 'colsample_by_tree': 0.798037114605478, 'colsample_by_level': 0.8809231215865033, 'colsample_by_node': 0.16955840508982306, 'bagging_fraction': 0.5512501653352012}. Best is trial 2 with value: 0.2044856218795681.
divvy_trips - INFO - Start Trial 4
divvy_trips - INFO - Performing split number 0


divvy_trips - INFO - Performing split number 1
divvy_trips - INFO - Performing split number 2
divvy_trips - INFO - Performing split number 3
divvy_trips - INFO - Performing split number 4
[I 2024-02-06 01:03:56,827] Trial 4 finished with value: 0.19505482761109055 and parameters: {'num_leaves': 9, 'max_depth': 8, 'n_estimators': 35, 'learning_rate': 0.37843940900146955, 'importance_type': 'split', 'subsample': 1, 'feature_fraction': 0.6605753400233634, 'colsample_by_tree': 0.507000070832764, 'colsample_by_level': 0.132989203577089, 'colsample_by_node': 0.7605783005531188, 'bagging_fraction': 0.9012817362748851}. Best is trial 4 with value: 0.19505482761109055.
divvy_trips - INFO - The best hyperparameters for <class 'lightgbm.sklearn.LGBMRegressor'> are:
divvy_trips - INFO - num_leaves:9
divvy_trips - INFO - max_depth:8
divvy_trips - INFO - n_estimators:35
divvy_trips - INFO - learning_rate:0.37843940900146955
divvy_trips - INFO - importance_type:split
divvy_trips - INFO - subsample:1


#### XGBRegressor

In [3]:
best_xgb_hyperparams_for_stops = \
  optimise_hyperparams(
    model_fn=XGBRegressor, 
    hyperparam_trials=5, 
    scenario="stop", 
    X=stop_table.drop("trips_next_hour", axis = 1),
    y=stop_table["trips_next_hour"]
)

divvy_trips - INFO - Beginning hyperparameter search
[I 2024-02-06 01:12:40,371] A new study created in memory with name: no-name-c5e3eea9-5666-4df8-8940-fc34d71b4ed1
divvy_trips - INFO - Start Trial 0
divvy_trips - INFO - Performing split number 0


/home/kobina/.cache/pypoetry/virtualenvs/src-Frx8CZzu-py3.10/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [01:12:41] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "colsample_by_level", "colsample_by_node", "colsample_by_tree" } are not used.

  warnings.warn(smsg, UserWarning)
divvy_trips - INFO - Performing split number 1
/home/kobina/.cache/pypoetry/virtualenvs/src-Frx8CZzu-py3.10/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [01:12:54] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "colsample_by_level", "colsample_by_node", "colsample_by_tree" } are not used.

  warnings.warn(smsg, UserWarning)
divvy_trips - INFO - Performing split number 2
/home/kobina/.cache/pypoetry/virtualenvs/src-Frx8CZzu-py3.10/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [01:13:16] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "colsample_by_level", "colsample_by_node", "colsample_by_tree" } are not used.

  warnings.warn(smsg, User

/home/kobina/.cache/pypoetry/virtualenvs/src-Frx8CZzu-py3.10/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [01:15:15] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "colsample_by_level", "colsample_by_node", "colsample_by_tree" } are not used.

  warnings.warn(smsg, UserWarning)
divvy_trips - INFO - Performing split number 1
/home/kobina/.cache/pypoetry/virtualenvs/src-Frx8CZzu-py3.10/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [01:15:25] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "colsample_by_level", "colsample_by_node", "colsample_by_tree" } are not used.

  warnings.warn(smsg, UserWarning)
divvy_trips - INFO - Performing split number 2
/home/kobina/.cache/pypoetry/virtualenvs/src-Frx8CZzu-py3.10/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [01:15:43] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "colsample_by_level", "colsample_by_node", "colsample_by_tree" } are not used.

  warnings.warn(smsg, User

/home/kobina/.cache/pypoetry/virtualenvs/src-Frx8CZzu-py3.10/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [01:17:11] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "colsample_by_level", "colsample_by_node", "colsample_by_tree" } are not used.

  warnings.warn(smsg, UserWarning)
divvy_trips - INFO - Performing split number 1
/home/kobina/.cache/pypoetry/virtualenvs/src-Frx8CZzu-py3.10/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [01:17:18] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "colsample_by_level", "colsample_by_node", "colsample_by_tree" } are not used.

  warnings.warn(smsg, UserWarning)
divvy_trips - INFO - Performing split number 2
/home/kobina/.cache/pypoetry/virtualenvs/src-Frx8CZzu-py3.10/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [01:17:29] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "colsample_by_level", "colsample_by_node", "colsample_by_tree" } are not used.

  warnings.warn(smsg, User

/home/kobina/.cache/pypoetry/virtualenvs/src-Frx8CZzu-py3.10/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [01:18:20] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "colsample_by_level", "colsample_by_node", "colsample_by_tree" } are not used.

  warnings.warn(smsg, UserWarning)
divvy_trips - INFO - Performing split number 1
/home/kobina/.cache/pypoetry/virtualenvs/src-Frx8CZzu-py3.10/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [01:18:26] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "colsample_by_level", "colsample_by_node", "colsample_by_tree" } are not used.

  warnings.warn(smsg, UserWarning)
divvy_trips - INFO - Performing split number 2
/home/kobina/.cache/pypoetry/virtualenvs/src-Frx8CZzu-py3.10/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [01:18:37] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "colsample_by_level", "colsample_by_node", "colsample_by_tree" } are not used.

  warnings.warn(smsg, User

/home/kobina/.cache/pypoetry/virtualenvs/src-Frx8CZzu-py3.10/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [01:19:28] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "colsample_by_level", "colsample_by_node", "colsample_by_tree" } are not used.

  warnings.warn(smsg, UserWarning)
divvy_trips - INFO - Performing split number 1
/home/kobina/.cache/pypoetry/virtualenvs/src-Frx8CZzu-py3.10/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [01:19:35] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "colsample_by_level", "colsample_by_node", "colsample_by_tree" } are not used.

  warnings.warn(smsg, UserWarning)
divvy_trips - INFO - Performing split number 2
/home/kobina/.cache/pypoetry/virtualenvs/src-Frx8CZzu-py3.10/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [01:19:46] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "colsample_by_level", "colsample_by_node", "colsample_by_tree" } are not used.

  warnings.warn(smsg, User